# Choose Your Own Adventure Workflow (Human In The Loop)

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

### Import RagaAI Catalyst components for tracing

In [2]:
from ragaai_catalyst.tracers import Tracer
from ragaai_catalyst import RagaAICatalyst, init_tracing
import os

catalyst = RagaAICatalyst(
    access_key=os.getenv("RAGAAI_CATALYST_ACCESS_KEY"),
    secret_key=os.getenv("RAGAAI_CATALYST_SECRET_KEY"),
    base_url=os.getenv("RAGAAI_CATALYST_BASE_URL"),
)

# Initialize tracer
tracer = Tracer(
    project_name="llamaindex_tracing_examples",
    dataset_name="human_in_the_loop",
    tracer_type="Agentic",
)

init_tracing(catalyst=catalyst, tracer=tracer)

INFO:httpx:HTTP Request: GET https://raw.githubusercontent.com/BerriAI/litellm/main/model_prices_and_context_window.json "HTTP/1.1 200 OK"


Token(s) set successfully


In [3]:
from typing import Any, List

from llama_index.llms.openai import OpenAI
from llama_index.core.bridge.pydantic import BaseModel, Field
from llama_index.core.prompts import PromptTemplate

In [4]:
class Segment(BaseModel):
    """Data model for generating segments of a story."""

    plot: str = Field(
        description="The plot of the adventure for the current segment. The plot should be no longer than 3 sentences."
    )
    actions: List[str] = Field(
        default=[],
        description="The list of actions the protaganist can take that will shape the plot and actions of the next segment.",
    )

In [5]:
SEGMENT_GENERATION_TEMPLATE = """
You are working with a human to create a story in the style of choose your own adventure.

The human is playing the role of the protaganist in the story which you are tasked to
help write. To create the story, we do it in steps, where each step produces a BLOCK.
Each BLOCK consists of a PLOT, a set of ACTIONS that the protaganist can take, and the
chosen ACTION. 

Below we attach the history of the adventure so far.

PREVIOUS BLOCKS:
---
{running_story}

Continue the story by generating the next block's PLOT and set of ACTIONs. If there are
no previous BLOCKs, start an interesting brand new story. Give the protaganist a name and an
interesting challenge to solve.


Use the provided data model to structure your output.
"""

In [6]:
FINAL_SEGMENT_GENERATION_TEMPLATE = """
You are working with a human to create a story in the style of choose your own adventure.

The human is playing the role of the protaganist in the story which you are tasked to
help write. To create the story, we do it in steps, where each step produces a BLOCK.
Each BLOCK consists of a PLOT, a set of ACTIONS that the protaganist can take, and the
chosen ACTION. Below we attach the history of the adventure so far.

PREVIOUS BLOCKS:
---
{running_story}

The story is now coming to an end. With the previous blocks, wrap up the story with a
closing PLOT. Since it is a closing plot, DO NOT GENERATE a new set of actions.

Use the provided data model to structure your output.
"""

In [7]:
import uuid
from typing import Optional

BLOCK_TEMPLATE = """
BLOCK
===
PLOT: {plot}
ACTIONS: {actions}
CHOICE: {choice}
"""


class Block(BaseModel):
    id_: str = Field(default_factory=lambda: str(uuid.uuid4()))
    segment: Segment
    choice: Optional[str] = None
    block_template: str = BLOCK_TEMPLATE

    def __str__(self):
        return self.block_template.format(
            plot=self.segment.plot,
            actions=", ".join(self.segment.actions),
            choice=self.choice or "",
        )

In [8]:
from llama_index.core.workflow import (
    Context,
    Event,
    StartEvent,
    StopEvent,
    Workflow,
    step,
)

In [9]:
class NewBlockEvent(Event):
    block: Block


class HumanChoiceEvent(Event):
    block_id: str

In [10]:
class ChooseYourOwnAdventureWorkflow(Workflow):
    def __init__(self, max_steps: int = 3, **kwargs):
        super().__init__(**kwargs)
        self.llm = OpenAI("gpt-4o")
        self.max_steps = max_steps

    @step
    async def create_segment(
        self, ctx: Context, ev: StartEvent | HumanChoiceEvent
    ) -> NewBlockEvent | StopEvent:
        blocks = await ctx.get("blocks", [])
        running_story = "\n".join(str(b) for b in blocks)

        if len(blocks) < self.max_steps:
            new_segment = self.llm.structured_predict(
                Segment,
                PromptTemplate(SEGMENT_GENERATION_TEMPLATE),
                running_story=running_story,
            )
            new_block = Block(segment=new_segment)
            blocks.append(new_block)
            await ctx.set("blocks", blocks)
            return NewBlockEvent(block=new_block)
        else:
            final_segment = self.llm.structured_predict(
                Segment,
                PromptTemplate(FINAL_SEGMENT_GENERATION_TEMPLATE),
                running_story=running_story,
            )
            final_block = Block(segment=final_segment)
            blocks.append(final_block)
            return StopEvent(result=blocks)

    @step
    async def prompt_human(
        self, ctx: Context, ev: NewBlockEvent
    ) -> HumanChoiceEvent:
        block = ev.block

        # get human input
        human_prompt = f"\n===\n{ev.block.segment.plot}\n\n"
        human_prompt += "Choose your adventure:\n\n"
        human_prompt += "\n".join(ev.block.segment.actions)
        human_prompt += "\n\n"
        human_input = input(human_prompt)

        blocks = await ctx.get("blocks")
        block.choice = human_input
        blocks[-1] = block
        await ctx.set("block", blocks)

        return HumanChoiceEvent(block_id=ev.block.id_)

In [11]:
import nest_asyncio

nest_asyncio.apply()

In [12]:
w = ChooseYourOwnAdventureWorkflow(timeout=None)

### Run the workflow with RagaAI Catalyst

In [13]:
with tracer:
    result = await w.run()

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



===
In the mystical land of Eldoria, a young adventurer named Arin stumbles upon an ancient map hidden within the pages of an old tome in the village library. The map hints at the location of the legendary Crystal of Eternity, said to grant the power to control time itself. Intrigued and eager for adventure, Arin decides to embark on a quest to find the crystal, but first, they must gather clues from the mysterious forest of Whispering Pines.

Choose your adventure:

Explore the Whispering Pines to find clues about the crystal's location.
Visit the village elder to learn more about the legend of the Crystal of Eternity.
Seek out a companion to join the quest for the crystal.



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



===
Arin decides to explore the Whispering Pines, a forest known for its ethereal beauty and enigmatic secrets. As Arin ventures deeper into the forest, they encounter a peculiar old tree with carvings that seem to tell a story of the crystal's past. Suddenly, a soft whispering voice fills the air, offering guidance but also warning of the dangers that lie ahead.

Choose your adventure:

Examine the carvings on the tree to decipher their meaning.
Follow the whispering voice deeper into the forest.
Set up camp and rest before continuing the journey.



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



===
Arin decides to examine the carvings on the tree, hoping to uncover more about the Crystal of Eternity. As they trace their fingers over the intricate symbols, a hidden compartment opens, revealing a small, glowing amulet. The amulet pulses with a gentle warmth, and Arin feels a strange connection to it, as if it holds a piece of the crystal's power.

Choose your adventure:

Take the amulet and continue exploring the forest.
Leave the amulet and follow the whispering voice deeper into the forest.
Set up camp and study the amulet's properties before proceeding.



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:ragaai_catalyst.tracers.agentic_tracing.utils.zip_list_of_unique_files: Zip file created successfully.
INFO:ragaai_catalyst.tracers.agentic_tracing.tracers.base: Traces saved successfully.


Uploading agentic traces...
Code already exists


In [14]:
final_story = "\n\n".join(b.segment.plot for b in result)
print(final_story)

In the mystical land of Eldoria, a young adventurer named Arin stumbles upon an ancient map hidden within the pages of an old tome in the village library. The map hints at the location of the legendary Crystal of Eternity, said to grant the power to control time itself. Intrigued and eager for adventure, Arin decides to embark on a quest to find the crystal, but first, they must gather clues from the mysterious forest of Whispering Pines.

Arin decides to explore the Whispering Pines, a forest known for its ethereal beauty and enigmatic secrets. As Arin ventures deeper into the forest, they encounter a peculiar old tree with carvings that seem to tell a story of the crystal's past. Suddenly, a soft whispering voice fills the air, offering guidance but also warning of the dangers that lie ahead.

Arin decides to examine the carvings on the tree, hoping to uncover more about the Crystal of Eternity. As they trace their fingers over the intricate symbols, a hidden compartment opens, revea